In [1]:
%load_ext line_profiler

from pathlib import Path
import sys

import numpy as np

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [2]:
import pandas as pd
import src.pipeline.rag as rag
import glob 
import os
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

# Create and clean directory
# from pathlib import Path
# Path("output/metric-rag").mkdir(parents=True, exist_ok=True)
# 
# files = glob.glob("output/metric-rag/*")
# for f in files:
#     os.remove(f)

## Loading saved comparator sets

In [3]:
raw_data = pd.read_parquet('output/comparator-sets/maintained_schools.parquet')
raw_comparators = pd.read_parquet('output/comparator-sets/academy_comparators.parquet')

In [4]:
df = raw_data

category_cols = df.columns.isin(rag.base_cols) | (df.columns.str.endswith("_Per Unit"))

df[df.columns[category_cols]].loc[100060]

Number of pupils                                                                                      0.0
OfstedRating (name)                                                                           Outstanding
Percentage SEN                                                                                        0.0
Percentage Free school meals                                                                         19.6
Total Internal Floor Area                                                                          1858.0
Age Average Score                                                                               55.792578
Teaching and Teaching support staff_Teaching staff_Per Unit                                           0.0
Teaching and Teaching support staff_Supply teaching staff_Per Unit                                    0.0
Teaching and Teaching support staff_Education support staff_Per Unit                                  0.0
Teaching and Teaching support staff_Agency sup

In [6]:
comps = raw_comparators.loc[105135]["Pupil"]
len(comps)

30

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [10]:
rs = rag.compute_rag(raw_data, raw_comparators)
pd.DataFrame(rs).set_index("URN")

,Category,SubCategory,Value,Mean,DiffMean,Key,PercentDiff,Percentile,Decile,RAG
URN,,,,,,,,,,
100000,Teaching and Teaching support staff,Teaching staff,4219.955498,2989.147021,1230.808477,outstanding,29.166385,92.333333,9,red
100000,Teaching and Teaching support staff,Supply teaching staff,45.340923,11.186708,34.154214,outstanding,75.327568,75.666667,7,amber
100000,Teaching and Teaching support staff,Education support staff,2329.561808,1383.424375,946.137433,outstanding,40.614395,92.333333,9,red
100000,Teaching and Teaching support staff,Agency supply teaching staff,266.307528,128.098011,138.209517,outstanding,51.898464,85.666667,8,amber
100000,Teaching and Teaching support staff,Educational consultancy,179.885793,134.315355,45.570438,outstanding,25.332984,65.666667,6,amber
...,...,...,...,...,...,...,...,...,...,...
100212,Other costs,Special facilities,0.000000,0.000000,0.000000,outstanding,0.000000,75.666667,7,amber
100212,Other costs,PFI charges,0.000000,0.000000,0.000000,outstanding,0.000000,99.000000,9,red
100212,Other costs,Interest charges for loan and bank,0.000000,0.000000,0.000000,outstanding,0.000000,99.000000,9,red


In [ ]:
%lprun -f rag.find_percentile list(rag.compute_rag(raw_data.head(10), raw_comparators))